# Comparing Models Performance

## Importing modules

In [ ]:
# Get the scripts from remote source
!git clone https://github.com/mlgomez0/Health_Therapist
!mv Health_Therapist/backend/ml_models/modules .
!mv Health_Therapist/backend/ml_models/datasets .
!mv Health_Therapist/backend/ml_models/LLMs_fine_tuning/phi3-qlora .
!mv Health_Therapist/requirements.txt .
!rm -rf Health_Therapist

Cloning into 'Health_Therapist'...
remote: Enumerating objects: 566, done.
remote: Counting objects: 100% (282/282), done.
remote: Compressing objects: 100% (222/222), done.
remote: Total 566 (delta 114), reused 147 (delta 54), pack-reused 284
Receiving objects: 100% (566/566), 25.74 MiB | 26.39 MiB/s, done.
Resolving deltas: 100% (204/204), done.
mv: cannot move 'Health_Therapist/backend/ml_models/modules' to './modules': Directory not empty
mv: cannot move 'Health_Therapist/backend/ml_models/datasets' to './datasets': Directory not empty


In [27]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF
from sklearn.pipeline import Pipeline
import numpy as np
from scipy import stats

In [ ]:
fine_tuned_phi3_model_responses = pd.read_csv('phi3-qlora/model_responses.csv', index_col=False)
original_phi3_model_responses = pd.read_csv('phi3-qlora/original_model_responses.csv', index_col=False)

In [ ]:
fine_tuned_phi3_model_responses.head()

,Unnamed: 0,input,output,therapist_output
0,0,i want a secure relationship with someone that...,It sounds like you're going through a really t...,good for you on your keen awareness of your di...
1,1,my boyfriend is in recovery from drug addictio...,I'm truly sorry to hear that you're going thro...,my empathy goes out to you relationships are t...
2,2,is this something i should be worried about sh...,I'm really glad you're reaching out and sharin...,it can be tricky to figure out if a child is t...
3,3,my friend is abusing her prescription medicine...,I'm really sorry to hear that you're going thr...,first of all where did you find the pills did ...
4,4,i terminated my counseling relationship with a...,I'm really glad to hear that you're considerin...,yes your former social worker should return yo...


## Topic Modeling

In [ ]:
text = fine_tuned_phi3_model_responses['input'] + ' ' + fine_tuned_phi3_model_responses['therapist_output']

In [ ]:
df = pd.read_csv('datasets/dataset.csv')

In [ ]:
vectorizer = CountVectorizer(min_df=10, max_df=0.5, stop_words="english") # To transform the text into a numerical representation
topic_model = NMF(n_components=10)

topic_pipeline = Pipeline(
    [
        ("vectorizer", vectorizer),
        ("topic_model", topic_model),
    ]
)

In [ ]:
topic_pipeline.fit(text)

Pipeline(steps=[('vectorizer',
                 CountVectorizer(max_df=0.5, min_df=10, stop_words='english')),
                ('topic_model', NMF(n_components=10))])

In [ ]:
# dictionary to map the topics

topics_map = {
    0: "Work-Life Balance",
    1: "Other",
    2: "Romantic Relationships",
    3: "Therapy and Therapist",
    4: "Negative Thoughts",
    5: "Parenthood",
    6: "People Feelings",
    7: "Anxiety and Depression",
    8: "Anxiety and Depression",
    9: "Counseling Issues"
}

In [ ]:
topics = topic_pipeline.transform(text)
topics = np.argmax(topics, axis=1)
topic_names = [topics_map[topic] for topic in topics]

In [ ]:
llm_eval = pd.read_csv('phi3-qlora/llm_eval.csv')
llm_eval.head()

,acorreal/phi3-mental-health,microsoft/Phi-3-mini-4k-instruct
0,7.5,7.5
1,8.5,7.5
2,7.5,7.5
3,7.5,7.5
4,8.5,7.5


In [ ]:
llm_eval['topic'] = topic_names

In [ ]:
llm_eval.head()

,acorreal/phi3-mental-health,microsoft/Phi-3-mini-4k-instruct,topic
0,7.5,7.5,Romantic Relationships
1,8.5,7.5,Romantic Relationships
2,7.5,7.5,Parenthood
3,7.5,7.5,People Feelings
4,8.5,7.5,Counseling Issues


In [26]:
# getting the tipics in the dataset
topics = llm_eval['topic'].unique()
topics

array(['Romantic Relationships', 'Parenthood', 'People Feelings',
       'Counseling Issues', 'Other', 'Anxiety and Depression',
       'Negative Thoughts', 'Therapy and Therapist', 'Work-Life Balance'],
      dtype=object)

In [49]:
def statistical_test(data, topics):
  result = []
  for topic in topics:
    data_t = data.loc[data['topic'] == topic]
    data1 = data_t['acorreal/phi3-mental-health']
    data2 = data_t['microsoft/Phi-3-mini-4k-instruct']
    data1 = data1.dropna()
    data2 = data2.dropna()
    _, p_value = stats.ttest_ind(data1, data2)
    result.append((topic, p_value))
  return result


In [50]:
results = statistical_test(llm_eval, topics)

/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


In [51]:
results

[('Romantic Relationships', 0.6637812137935639),
 ('Parenthood', 0.5588598585998714),
 ('People Feelings', 0.3666797221730457),
 ('Counseling Issues', 0.7803069318544259),
 ('Other', 0.5389536342821932),
 ('Anxiety and Depression', 0.6610969069607628),
 ('Negative Thoughts', 0.3757297427948375),
 ('Therapy and Therapist', 0.3757297427948375),
 ('Work-Life Balance', nan)]

In [57]:
llm_t_test = llm_eval.groupby('topic').mean()

In [55]:
llm_t_test['p_value'] = 0.0
for result in results:
  llm_t_test.loc[result[0], 'p_value'] = result[1]

In [56]:
llm_t_test

,acorreal/phi3-mental-health,microsoft/Phi-3-mini-4k-instruct,p_value
topic,,,
Anxiety and Depression,7.573529,7.691176,0.661097
Counseling Issues,7.788462,7.740000,0.780307
Negative Thoughts,7.833333,6.583333,0.375730
Other,6.428571,7.142857,0.538954
Parenthood,7.633333,7.566667,0.558860
People Feelings,7.700000,7.560000,0.366680
Romantic Relationships,7.529412,7.617647,0.663781
Therapy and Therapist,7.833333,6.583333,0.375730
Work-Life Balance,7.500000,7.500000,NaN
